In [5]:
%%capture
!pip install gdown

In [ ]:
import gdown
import os
import pandas as pd
os.makedirs("data", exist_ok=True)
file_id = "1qqxV6PRtKMhtPm9ysICi7k4jNijj4Jql"  
url = f"https://drive.google.com/uc?id={file_id}"
output = "data/marked_sentences_472_cutted.csv"
gdown.download(url, output, quiet=False)
slavery = pd.read_csv(output)
slavery.head(5)

Downloading...
From: https://drive.google.com/uc?id=1qqxV6PRtKMhtPm9ysICi7k4jNijj4Jql
To: c:\Users\1\Downloads\hidden slavery\3. classification\data\marked_sentences_472_cutted.csv
100%|██████████| 56.0M/56.0M [00:20<00:00, 2.79MB/s]


,case_number,text
0,1-498/2023 (1-1222/2022;),"ФИО11 совершила деяние, направленные на органи..."
1,1-115/2023,Подсудимая Сарапкина Е.А. в <адрес> совершила ...
2,1-306/2017,Подсудимая Сузый ФИО14. совершила организацию ...
3,1-27/2019 (1-346/2018;),"Чугунова О.И. совершила деяния, направленные н..."
4,2-19/2012,Волков А.С. принимал участие в преступном сооб...
...,...,...
467,1-333/2011,"Рензяев А.П., Донской А.А., Медведков А.В., Ан..."
468,1-10/2020,"Тумаков С.В. совершил угрозу убийством, и имел..."
469,1-240/2012,"Малахов В.С. незаконно лишил человека свободы,..."
470,1-143/2022,В период с 20.00 час. ДД.ММ.ГГГГ до 12.25 час....


Или:

In [ ]:
file_id = "1vaza37kFjcolQE8tJ5pXtqx2BtbwCOT2"  
url = f"https://drive.google.com/uc?id={file_id}"
output = "data/unmarked_sentences_cutted.csv"
gdown.download(url, output, quiet=False)
all_cases = pd.read_csv(output)
all_cases.head(5)

Downloading...
From (original): https://drive.google.com/uc?id=1vaza37kFjcolQE8tJ5pXtqx2BtbwCOT2
From (redirected): https://drive.google.com/uc?id=1vaza37kFjcolQE8tJ5pXtqx2BtbwCOT2&confirm=t&uuid=85f1bae6-e84c-474b-a0c2-8527f3e68d93
To: c:\Users\1\Downloads\hidden slavery\3. classification\data\unmarked_sentences_cutted.csv
100%|██████████| 259M/259M [05:58<00:00, 724kB/s]  


,case_number,text
0,1-687/2012,Загитов А.С. совершил преступление против здор...
1,1-459/2012,"Карпова Т.А. совершила действия, направленные ..."
2,1-285/2012,"Салагаева Рита Кертибиевна совершила деяния, н..."
3,1-447/2015,ПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nДД.ММ.Г...
4,1-366/2015,"Воробьева В.С. обвиняется в совершении деяний,..."
...,...,...
2499,1-66/2020 (1-483/2019;),"ФИО1 совершил вымогательство, то есть требован..."
2500,1-5/2020 (1-105/2019;),"Титова В.А совершил похищение человека, с прим..."
2501,1-18/2020 (1-291/2019;),"Хамидзода Ф.Д., Абдуллозода Н.Н., Назаров С., ..."
2502,1-7/2020,Подсудимый Осьмаков А.Н. незаконно проник в жи...


In [ ]:
import pandas as pd
import google.generativeai as genai
import json
import time
import math


genai.configure(api_key="ВАШ_АПИ_КЛЮЧ")

# Функция для оценки количества токенов
def estimate_tokens(text):
    return math.ceil(len(text) / 3)  # Оценка: 1 токен ≈ 3 символа

# Функция для обработки батча
def classify_batch(batch):
    try:
        prompt = """ТЕКСТ ИНСТРУКЦИИ
        Проанализируй следующие приговоры и верни JSON в формате:
        [
            {{"case_number": "<номер дела>", "slavery_detected": true/false, "reason": "<обоснование>", "victim": <количество жертв (если есть, иначе 0)>}},
            ...
        ]

        **Приговоры**:
        """

        for case_number, text in batch:
            prompt += f"\n\nНомер дела: {case_number}\nТекст: {text}\n---"

        model = genai.GenerativeModel("gemini-2.5-pro-exp-03-25")
        response = model.generate_content(prompt)

        response_text = response.text.strip()
        print(f"Ответ модели:\n{response_text}")

        return safe_json_parse(response_text)

    except Exception as e:
        print(f"Ошибка обработки batch {batch}: {e}")
        return [{"case_number": case_number, "slavery_detected": None, "reason": "Ошибка обработки", "victim": 0} for case_number, _ in batch]

# Функция для парсинга JSON
def safe_json_parse(response_text):
    try:
        response_text = response_text.strip("```json").strip("```").strip()
        return json.loads(response_text)
    except json.JSONDecodeError:
        print(f"Ошибка декодирования JSON: {response_text}")
        return []

if "slavery" not in locals():
    raise ValueError("slavery не определен!")

slavery = slavery.drop_duplicates()

if "text" not in slavery.columns:
    raise ValueError("Файл не содержит колонку 'text'!")

# Оцениваем количество токенов в каждом приговоре
slavery["num_tokens"] = slavery["text"].apply(estimate_tokens)

# Разбиваем приговоры на батчи по ≤ 200K токенов
batch = []
current_tokens = 0
batches = []

for _, row in slavery.iterrows():
    case_number, text, tokens = row["case_number"], row["text"], row["num_tokens"]
    
    if current_tokens + tokens > 195000:
        batches.append(batch)
        batch = []
        current_tokens = 0
    
    batch.append((case_number, text))
    current_tokens += tokens

if batch:
    batches.append(batch)

results = []
for batch in batches:
    batch_results = classify_batch(batch)
    results.extend(batch_results)

    print("Ожидание 10 секунд перед следующим запросом...")
    time.sleep(10)

results_df = pd.DataFrame(results)
results_df.to_csv("тест1.csv", index=False)

print("Классификация завершена! Результат сохранен в 'тест1.csv'")


In [ ]:
file_id = "1jmlDZaHKqHoNtffIPc34Q3er4bnoM5Ai"  
url = f"https://drive.google.com/uc?id={file_id}"
output = "data/gemini25_version3.csv"
gdown.download(url, output, quiet=False)
slavery_classified = pd.read_csv(output)
slavery_classified.head(5)


In [ ]:
#Выполнение этой ячейки необходимо, если результат классификации ИИ не был предварительно соединен с размеченным датафреймом
df_merged = slavery.merge(slavery_classified, on=['case_number'], how='left', indicator=True)

In [ ]:
value_counts = df_merged['slavery_detected'].value_counts()
print(value_counts)

non_na_rows = df_merged[df_merged['slavery_detected'].notna()]
len(non_na_rows)


In [ ]:
df_merged = df_merged.dropna(subset=['slavery_detected'])
df_merged['slavery_detected'] = df_merged['slavery_detected'].map({True: 1, False: 0})
#slavery['slavery_detected'] = slavery['slavery_detected'].fillna(0).astype(int)


In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns


y_true = df_merged['Slavery_1_step']  # Ручная разметка
y_pred = df_merged['slavery_detected']  # Разметка ИИ

# Рассчитываем ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Выводим Classification Report для классификации 1
print("\nClassification Report - slav_1:")
print(classification_report(y_true, y_pred))

# Рассчитываем взвешенную F1-меру вручную
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\nWeighted F1 Score: {weighted_f1:.4f}")

# Рассчитываем macro-average F1-меру вручную
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\nMacro-average F1 Score: {macro_f1:.4f}")

# Выводим матрицу ошибок для slav_1
cm_1 = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix - slav_2:")
print(cm_1)

# Визуализируем матрицу ошибок
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

sns.heatmap(cm_1, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'], ax=axes[0])
axes[0].set_title('Confusion Matrix - slav_2')
axes[0].set_xlabel('Predicted')
axes
